In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import glob
from pathlib import Path

pd.set_option("display.max_columns", None)

In [6]:
df = pd.read_parquet("D:/FSS/OneDrive - Luiss/DATA LAB/WIH/WIH/data/report/EU_skill_per_occupation_oc3_2024_T4.parquet")

In [7]:
df.head()

,territory,first_active_year,quarter,occupation3d_id,skill_id,n_oja
0,EU,2024,4,OC261,ESCOv1_6453,64
1,EU,2024,4,OC261,ESCOv1_3393,1
2,EU,2024,4,OC261,ESCOv1_6364,16
3,EU,2024,4,OC261,ESCOv1_2955,23
4,EU,2024,4,OC261,ESCOv1_10715,1


# Append di tutti i parquet

In [151]:
%%time

def merge_parquet_files(directory_path):
    """
    Legge tutti i file parquet in una cartella specifica e li unisce in un unico DataFrame.
    
    Args:
        directory_path (str): Percorso della cartella contenente i file parquet
        
    Returns:
        pandas.DataFrame: DataFrame contenente i dati di tutti i file parquet uniti
    """
    # Assicurati che il percorso termini con una barra
    directory_path = os.path.join(directory_path, '')
    
    # Trova tutti i file parquet nella cartella
    parquet_files = glob.glob(directory_path + "OC*.parquet")
    
    if not parquet_files:
        print(f"Nessun file parquet trovato nella cartella: {directory_path}")
        return None
    
    # Lista per memorizzare i DataFrame
    dfs = []
    
    # Leggi ogni file parquet e aggiungilo alla lista
    for file in parquet_files:
        try:
            print(f"Lettura del file: {file}")
            df = pd.read_parquet(file)
            if df.query("occupation3d_id.isnull()==True", engine='python').shape[0]>0:
                print("valori null:", df.query("occupation3d_id.isnull()==True", engine='python').shape[0])
            dfs.append(df)
        except Exception as e:
            print(f"Errore nella lettura del file {file}: {e}")
    
    if not dfs:
        print("Nessun file parquet valido trovato.")
        return None
    
    # Concatena tutti i DataFrame in un unico DataFrame
    merged_df = pd.concat(dfs, ignore_index=True)
    
    print(f"Numero totale di file uniti: {len(dfs)}")
    print(f"Dimensione del DataFrame finale: {merged_df.shape}")
    
    return merged_df


# Elimino prima il file df_oja se esiste
# Rimuove il file se esiste
file_path = "./data/db_target_it/df_oja.parquet"
if os.path.exists(file_path):
    os.remove(file_path)

df_oja = merge_parquet_files("./data/db_target_it").drop(columns=['skill_hier3_id', 'skill_hier2_id', 'skill_hier1_id', 'skill_hier0_id'
                                                                  # , 'skill_desc', 'skill_type', 'reuse_type', 'f_obsolete'
                                                                  # , 'f_digicomp', 'f_green', 'f_ict', 'f_language', 'skill_h3'
                                                                  # , 'skill_h2', 'skill_h1', 'skill_h0'
                                                                  # , 'percorso', 'facolta'
                                                                  # , 'corso_studi_id', 'corso_studi_desc', 'cp2021', 'cp2021_desc'
                                                                  # , 'occupation3d'                                                      
                                                                 ])
df_oja.to_parquet(file_path)

Lettura del file: ./data/db_target_it\OC231_2023.parquet
Lettura del file: ./data/db_target_it\OC231_2024.parquet
Lettura del file: ./data/db_target_it\OC241_2023.parquet
Lettura del file: ./data/db_target_it\OC241_2024.parquet
Lettura del file: ./data/db_target_it\OC242_2023.parquet
Lettura del file: ./data/db_target_it\OC242_2024.parquet
Lettura del file: ./data/db_target_it\OC243_2023.parquet
Lettura del file: ./data/db_target_it\OC243_2024.parquet
Lettura del file: ./data/db_target_it\OC251_2023_1.parquet
Lettura del file: ./data/db_target_it\OC251_2023_2.parquet
Lettura del file: ./data/db_target_it\OC251_2024.parquet
Lettura del file: ./data/db_target_it\OC252_2023.parquet
Lettura del file: ./data/db_target_it\OC252_2024.parquet
Lettura del file: ./data/db_target_it\OC261_2023.parquet
Lettura del file: ./data/db_target_it\OC261_2024.parquet
Lettura del file: ./data/db_target_it\OC263_2023.parquet
Lettura del file: ./data/db_target_it\OC263_2024.parquet
Lettura del file: ./data/db

In [152]:
print("n. righe:", df_oja.shape[0])
print("n. oja:", df_oja.oja_id.nunique())

n. righe: 2417983
n. oja: 123950


# Lookup Tables

## ESCOSKILLS

In [154]:
lk_escoskills = (pd.read_excel("./data/codelist_escoskill_v111.xlsx")
                 [['skill_id', 'esco_v0101_description', 'esco_v0101_skillstype', 'esco_v0101_reusetype', 'esco_v0101_obsolete'
                  , 'esco_v0101_digcomp', 'esco_v0101_green', 'esco_v0101_ict', 'esco_v0101_language'
                   , 'esco_v0101_hier_label_3', 'esco_v0101_hier_label_2', 'esco_v0101_hier_label_1', 'esco_v0101_hier_label_0']]
                 .rename(columns={'skill_id': 'skill_id'
                                  , 'esco_v0101_description': 'skill_desc'
                                  , 'esco_v0101_skillstype': 'skill_type'
                                  , 'esco_v0101_reusetype': 'reuse_type'
                                  , 'esco_v0101_obsolete': 'f_obsolete'
                                  , 'esco_v0101_digcomp': 'f_digicomp'
                                  , 'esco_v0101_green': 'f_green'
                                  , 'esco_v0101_ict': 'f_ict'
                                  , 'esco_v0101_language': 'f_language'
                                  , 'esco_v0101_hier_label_3': 'skill_h3'
                                  , 'esco_v0101_hier_label_2': 'skill_h2'
                                  , 'esco_v0101_hier_label_1': 'skill_h1'
                                  , 'esco_v0101_hier_label_0': 'skill_h0'
                                 })
                )
lk_escoskills.to_parquet("./data/db_target_it/lk_escoskills.parquet")
print("n. righe:", lk_escoskills.shape[0])
lk_escoskills.head(3)

n. righe: 13989


,skill_id,skill_desc,skill_type,reuse_type,f_obsolete,f_digicomp,f_green,f_ict,f_language,skill_h3,skill_h2,skill_h1,skill_h0
0,ESCOv1_4545,"Exchanging and conveying information, ideas, c...",knowledge,cross-sector,N,N,N,N,N,personal skills and development,personal skills and development,generic programmes and qualifications,knowledge
1,ESCOv1_11185,Understand and analyse the behaviour of specif...,knowledge,sector-specific,N,N,N,N,N,personal skills and development,personal skills and development,generic programmes and qualifications,knowledge
2,ESCOv1_321,The techniques and methods used to improve awa...,knowledge,cross-sector,N,N,N,N,N,personal skills and development,personal skills and development,generic programmes and qualifications,knowledge


## OFFERTA LUISS

In [155]:
offerta = pd.read_excel("./data/offerta_luiss/offerta_formativa_luiss.xlsx")
offerta.columns = ['percorso', 'facolta', 'corso_studi_id', 'corso_studi_desc', 'cp2021', 'cp2021_desc']
print("Numero di righe del file:", offerta.shape[0])
print("cp2021 univoci:", offerta.cp2021.nunique())


# Raccordo Istat-Isco3
raccordo_istat_isco3 = pd.read_excel("./data/classificazioni/raccordo-ISCO08_III-Cp2021_V.xlsx", dtype=str)
raccordo_istat_isco3.columns = ['occupation3d_id', 'occupation3d', 'cp2021', 'cp2021_desc']
raccordo_istat_isco3['occupation3d_id'] = 'OC' + raccordo_istat_isco3['occupation3d_id']

offerta_isco08_liv3 = offerta.merge(raccordo_istat_isco3[['occupation3d_id', 'occupation3d', 'cp2021']], how='inner', on='cp2021')
offerta_isco08_liv3.to_parquet("./data/db_target_it/lk_offerta_isco3.parquet")

Numero di righe del file: 118
cp2021 univoci: 54


# TO DO

Per corso di laurea, per ogni professione calcolare il numero di annunci e il numero di skill per trimestre (o la % di annunci the prevedono la skill, calcolata per tutte le skill)

Calcolarlo sia a livello ISCO3, sia utilizzando la riclassificazione ISCO4

# STATISTICHE

## Skill per occupation id

In [165]:
%%time

def base_skill_per_occ(df, oc_level='occupation3d_id'):

    df_oja_oc = (df
                  .query("skill_id.fillna('')!=''", engine='python')
                  .groupby([oc_level, 'skill_id'], dropna=False)
                  .agg({'oja_id': 'nunique'}).reset_index()
                  .rename(columns={'oja_id': 'n_oja'})
                  .sort_values(by=[oc_level, 'n_oja'], ascending=[True, False])
                  .merge(lk_escoskills, how='left', on='skill_id')
                 )
    n_start = df_oja_oc.shape[0]

    # Aggiungo ranking Totale
    df_oja_oc['rank_tot'] = (df_oja_oc.groupby([oc_level]).cumcount() + 1)

    # Calcolo ranking differenziati per skill_type
    df_oja_oc['rank_skill_type'] = (df_oja_oc
                                     .sort_values(by=[oc_level, 'skill_type', 'n_oja'], ascending=[True, True, False])
                                     .groupby([oc_level, 'skill_type']).cumcount() + 1)

    # Calcolo ranking competenze ICT
    df_oja_oc_ict = (df_oja_oc.query("f_ict=='Y'")
                      [[oc_level, 'skill_id', 'n_oja']]
                      .sort_values(by=[oc_level, 'n_oja'], ascending=[True, False])
                     )
    df_oja_oc_ict['rank_ict'] = (df_oja_oc_ict.groupby([oc_level]).cumcount() + 1)

    # Calcolo ranking competenze GREEN
    df_oja_oc_green = (df_oja_oc.query("f_green=='Y'")
                        [[oc_level, 'skill_id', 'n_oja']]
                        .sort_values(by=[oc_level, 'n_oja'], ascending=[True, False])
                       )
    df_oja_oc_green['rank_green'] = (df_oja_oc_green.groupby([oc_level]).cumcount() + 1)

    # Merge di tutti i rank
    df_oja_oc = (df_oja_oc
                  .merge(df_oja_oc_ict.drop(columns=['n_oja']), how='left', on=[oc_level, 'skill_id'])
                  .merge(df_oja_oc_green.drop(columns=['n_oja']), how='left', on=[oc_level, 'skill_id'])
                 )
    if n_start!=df_oja_oc.shape[0]:
        print("ATTENZIONE NUMERO DI RIGHE CAMBIATO!")
    
    return df_oja_oc


########################################################################################################################à

def stat_skill_per_occ(df, oc_level='occupation3d_id', out_file=''):

    # Cacolo numero di OJA per professione
    n_oja_prof = (df
                  .groupby([oc_level], dropna=False)
                  .agg({'oja_id': 'nunique'}).reset_index()
                  .rename(columns={'oja_id': 'n_oja'})
                 )
    # Il dataset ora viene "sdoppiato" per corso di laurea, dal momento che la stessa professione può essere presente anche in altri corsi
    n_oja_prof_luiss = (offerta_isco08_liv3
                       .merge(n_oja_prof, how='left', on=oc_level)
                       .sort_values(by=['percorso', 'facolta', 'corso_studi_id', 'corso_studi_desc', 'cp2021']
                                    , ascending=[True, True, True, True, True])

                      )
    
    
    # Definisco tabella di base per calcolo statistiche
    df_oja_oc = base_skill_per_occ(df=df, oc_level=oc_level)
    
    


    # Il dataset ora viene "sdoppiato" per corso di laurea, dal momento che la stessa professione può essere presente anche in altri corsi
    df_oja_oc_luiss = (offerta_isco08_liv3
                       .merge(df_oja_oc, how='left', on=oc_level)
                       .sort_values(by=['percorso', 'facolta', 'corso_studi_id', 'corso_studi_desc', 'cp2021', 'n_oja']
                                    , ascending=[True, True, True, True, True, False])

                      )

    # Calcolo top 10 skill per corso e professione
    df_oja_oc_luiss_rank_tot_10 = df_oja_oc_luiss.query("rank_tot<=10").drop(columns={'rank_skill_type', 'rank_ict', 'rank_green'})
    df_oja_oc_luiss_rank_skilltype_10 = (df_oja_oc_luiss.query("rank_skill_type<=10").drop(columns={'rank_ict', 'rank_green'})
                                         .sort_values(by=['percorso', 'facolta', 'corso_studi_id', 'corso_studi_desc', 'cp2021', 'skill_type', 'n_oja']
                                                      , ascending=[True, True, True, True, True, True, False])
                                        )
    df_oja_oc_luiss_rank_ict_10 = df_oja_oc_luiss.query("rank_ict<=10").drop(columns={'rank_skill_type', 'rank_green'})
    df_oja_oc_luiss_rank_green_10 = df_oja_oc_luiss.query("rank_green<=10").drop(columns={'rank_skill_type', 'rank_ict'})

    # Export in excel
    with pd.ExcelWriter(out_file) as writer:
        n_oja_prof_luiss.to_excel(writer, sheet_name='n. oja', index=False)
        df_oja_oc_luiss_rank_tot_10.to_excel(writer, sheet_name='Top 10', index=False)
        df_oja_oc_luiss_rank_skilltype_10.to_excel(writer, sheet_name='Top 10 skill type', index=False)
        df_oja_oc_luiss_rank_ict_10.to_excel(writer, sheet_name='Top 10 ICT', index=False)
        df_oja_oc_luiss_rank_green_10.to_excel(writer, sheet_name='Top 10 green', index=False)
        
        
# ESECUZIONE
# Statistiche 3° trimestre 2024
stat_skill_per_occ(df=df_oja.query("first_active_year==2024 and first_active_month>=7 and first_active_month<=9")
                   , oc_level='occupation3d_id'
                   , out_file='./data/report/skill_per_professione_oc3_2024_T3.xlsx')


Wall time: 1.86 s


In [162]:
df_oja_oc = base_skill_per_occ(df=df_oja, oc_level='occupation3d_id')

In [164]:
df_oja.head()

,oja_id,first_active_date,first_active_month,first_active_year,last_active_date,language,occupation4d_id,occupation3d_id,skill_id,city_id,nuts3_id,country_id,contract_id,education_id,economic_activity2d_id,economic_activity1d_id,activity_sector_id,salary_id,working_time_id,experience_id,source_id,source_category_id
0,1156831726,2023-05-23,5,2023,2023-09-20,en,OC2310,OC231,ESCOv1_10212,,,IT,LTD,ED6,,C,C,KE6-12,,,637,JOBSE
1,1111864855,2023-02-13,2,2023,2023-06-13,en,OC2310,OC231,ESCOv1_6502,,,IT,,ED6,,Q,G-U,,,Y2-4,637,JOBSE
2,1187654550,2023-08-07,8,2023,2023-08-29,en,OC2310,OC231,ESCOv1_1678,015146,ITC4C,IT,NLTD,ED6,M72,M,G-U,,,,390,EMPAG_WEB
3,1141644274,2023-04-18,4,2023,2023-08-16,en,OC2310,OC231,ESCOv1_1976,018022,ITC48,IT,,ED6,P85,P,G-U,,,Y_LT1,390,EMPAG_WEB
4,1107283740,2023-02-01,2,2023,2023-06-01,en,OC2310,OC231,ESCOv1_6747,,,IT,LTD,ED6,M71,M,G-U,,FT,Y2-4,637,JOBSE
